In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.5.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools PyCall PyPlot Knet HDF5 AutoGrad"
JULIA_PACKAGES_IF_GPU="CUDA"
JULIA_NUM_THREADS=4
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"'
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi


echo -e "function KnetArray(x::CuArray{T,N}) where {T,N}\n\tp = Base.bitcast(Knet.Cptr, pointer(x))\n\tk = KnetPtr(p, sizeof(x), Int(CUDA.device().handle), x)\n\tKnetArray{T,N}(k, size(x))\nend" >> /root/.julia/packages/Knet/OYNCT/src/knetarrays/karray.jl

Installing Julia 1.5.2 on the current Colab Runtime...
2020-11-27 18:14:42 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.5/julia-1.5.2-linux-x86_64.tar.gz [105324048/105324048] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
 Installing known registries into `~/.julia`
######################################################################## 100.0%
      Added registry `General` to `~/.julia/registries/General`
  Resolving package versions...
  Installed VersionParsing ── v1.2.0
  Installed Artifacts ─────── v1.3.0
  Installed ZMQ ───────────── v1.2.1
  Installed MbedTLS ───────── v1.0.3
  Installed ZeroMQ_jll ────── v4.3.2+5
  Installed MbedTLS_jll ───── v2.16.8+1
  Installed Parsers ───────── v1.0.12
  Installed Conda ─────────── v1.5.0
  Installed IJulia ────────── v1.23.0
  Installed JSON ──────────── v0.21.1
  Installed SoftGlobalScope ─ v1.1.0
  Installed JLLWrappers ───── v1.1.3
########################################################################

# How to run?

First know that this is a one-script notebook to run the entire project. 


1.   Run upper cell. (It install julia 1.5.2)
2.   Restart the notebook. (This allows you to use julia language)
3.   






In [14]:
versioninfo()

Julia Version 1.5.2
Commit 539f3ce943 (2020-09-23 23:17 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, broadwell)
Environment:
  JULIA_NUM_THREADS = 4


In [2]:
using Knet
using AutoGrad
using HDF5
using PyCall
using CUDA; aType=(CUDA.functional() ? KnetArray{Float32} : Array{Float32})

┌ Info: Precompiling Knet [1902f260-5fb4-5aff-8c31-6271790ab950]
└ @ Base loading.jl:1278
  ** incremental compilation may be fatally broken for this module **



######################################################################### 100.0%


######################################################################### 100.0%


######################################################################### 100.0%


KnetArray{Float32,N} where N

In [ ]:
CUDA.functional()

true

# DATA

Note that you should upload the DATA.hdf5.gz file to system each time you run, because it is recycled each time.

In [8]:
run(`gunzip DATA.hdf5.gz`)

Process(`gunzip DATA.hdf5.gz`, ProcessExited(0))

In [9]:
fid = h5open("DATA.hdf5", "r")

xTrn = read(fid["trn"]["x"]);
xTst = read(fid["tst"]["x"]);

yTrn = [read(fid["trn"]["y"][string(i)]) for i in 1:28];
yTst = [read(fid["tst"]["y"][string(i)]) for i in 1:28];

heads = read(fid["heads"]);


14-element Array{String,1}:
 "22RV1.AR-C19"
 "22RV1.AR-V7"
 "LNCaP.dht.AR"
 "LNCaP.veh.AR"
 "22RV1.HOXB13"
 "LN95.AR-C19"
 "LN95.AR-V7"
 "LN95.HOXB13"
 "malignant.1.AR"
 "malignant.2.AR"
 "malignant.3.AR"
 "malignant.4.AR"
 "non-malignant.1.AR"
 "non-malignant.2.AR"

In [10]:
summary.([xTrn, yTrn, xTst, yTst])

4-element Array{String,1}:
 "1000×4×1×13687 Array{Float32,4}"
 "28-element Array{Array{Float32,4},1}"
 "1000×4×1×13688 Array{Float32,4}"
 "28-element Array{Array{Float32,4},1}"

In [3]:


# aType = Array{Float32}
function minibatch(
    x, y; batchsize::Int32=100, shuffle::Bool=false, atype::Type=Array{Float32})
    """
    This function converts x and y data. Then initiates and returns Data type object.
    """
    etype = eltype(atype)
    x = aType(x)
    y = [atype(yi) for yi in y]
    n = size(x)[end]
    Data(x,y, batchsize, shuffle, n, 1:n)
end


mutable struct Data
    x
    y
    batchsize::Int32
    shuffle::Bool
    n::Int32
    indices::Vector{Int32}
end

import Base: length, iterate, eltype, HasEltype

function length(d::Data)
    # Your code here
    Int(ceil(d.n / d.batchsize))
end



function iterate(d::Data, state=(0)) # here the start point of state is 0
    if state == 0 && d.shuffle
        d.indices = randperm(d.n)
    end

    if state >= d.n
        return nothing
    end
    i = state + 1 # here the beginning of the current slice
    j = state + d.batchsize # here the end of the current slice

    idx = d.indices[i:j]
    xbatch = d.x[:,:,:,i:j]
    ybatch = [y[:,:,:,i:j] for y in d.y]

    return ((xbatch, ybatch), j) # here it returns the batch and the next state of the iteration
end



iterate (generic function with 337 methods)

In [11]:
dTrn = minibatch(xTrn, yTrn; batchsize=Int32(100), atype=aType);
dTst = minibatch(xTst, yTst; batchsize=Int32(100), atype=aType);

In [13]:
Knet.Cptr

Ptr{Nothing}

# Losses

In [4]:


function aproxLogFact(n)
    """
    Approximation for log n!
    """
    Float32(n * log(n) - n  + log(n * (1 + 4n * (1 + 2n)))/6 + log(π)/2)
end


function multinomial_nll(ypred, ytrue)
    pV = ypred ./ sum(ypred, dims=1)
    logPV = log.(pV)
    pS = sum(logPV .* ytrue, dims=1)
    n = sum(ytrue, dims=1)
    nS = (aproxLogFact.(n) .- sum(aproxLogFact.(ytrue), dims=1))
    logP = pS .+ nS
    loss = sum(logP) / prod(size(logP))
end

function mse(ypred, ytrue)
    λ = Float32(0.5) * sum(ypred) / size(ypred, 1)
    C = λ * abs2.(log.(1 .+ sum(ypred)) .- log.(1 .+ ytrue)) ./ size(ypred, 1)
    sum(C) / prod(size(C))
end


mse (generic function with 1 method)

# Layers

In [5]:
mutable struct Conv
    w
    b
    f
    pDrop
    padding
    dilation
    nParameters
end



(c::Conv)(x) =
    c.f.(
        conv4(
            c.w,
            dropout(x, c.pDrop),
            padding=c.padding,
            dilation=c.dilation
        ) .+ c.b
    )



Conv(w1::Int, w2::Int, cx::Int, cy::Int;
    f=
        relu,
    pDrop=
        0,
    padding=
        0,
    dilation=
        1,
    xType=
        Array{Float32},
    scale=
        0.01
    ) = Conv(
            Param(xType(rand(w1,w2,cx,cy)) .* eltype(xType)(scale), Adam()),
            Param(xType(zeros(1,1,cy,1)), Adam()),
            f,
            pDrop,
            padding,
            dilation,
            (w1*w2*cx +1)*cy + cy
        )

################################################################################
struct Pool
    wSize
    nParameters
end

(p::Pool)(x) = pool(x; window=p.wSize, mode=1)
Pool(x) = Pool(x, 0)
################################################################################
mutable struct BatchNorm
    moments
    params
    nParameters
end

(b::BatchNorm)(x) = batchnorm(x, b.moments, b.params)

BatchNorm(C) = BatchNorm(bnmoments(), arrayType(bnparams(C)), 0)

################################################################################

struct Dense
    w
    b
    f
    pDrop
    window
    nParameters
end

Dense(i,o;
    f=
        identity,
    window=
        1,
    xType=
        Array{Float32},
    scale=
        0.01,
    pDrop=
        0
    ) =
        Dense(
            Param(xType(rand(o,i)) .* eltype(xType)(scale), Adam()),
            Param(xType(zeros(o)), Adam()),
            f,
            pDrop,
            window,
            (i+1)*o + o
        )

(d::Dense)(x) =
    d.f.(
        d.w *
            mat(
                dropout(x, d.pDrop),
            ) .+ d.b
        )


################################################################################


# Define a deconvolution layer:

struct DeConv
    w
    b
    padding
    nParameters
end


DeConv(w1::Int, w2::Int, cx::Int, cy::Int;
    padding=
        0,
    xType=
        Array{Float32},
    scale=
        0.01
    ) = DeConv(
            Param(xType(rand(w1,w2,cx,cy)) .* eltype(xType)(scale), Adam()),
            Param(xType(zeros(1,1,cx,1)), Adam()),
            padding,
            (w1*w2*cx)*cy + cx
        )



(dC::DeConv)(x) =
    deconv4(
        dC.w,
        x,
        padding=dC.padding
    ) .+ dC.b
################################################################################


# Architecture

In [6]:

# Define a body (up to bottleneck)
mutable struct Body
    layers
    Body(layers...) = new(layers)
end


function addLayer!(b::Body, l)
    b.layers = (
        b.layers...,
        l
    )
end


(b::Body)(x) = (for l in b.layers;x = l(x);end;x)


##################################

# Define Head structure

mutable struct Head
    layers
    lossF
    Head(layers...; lossF=identity) = new(layers, lossF)
end

(h::Head)(x) = (for l in h.layers;x = l(x);end;x)



###########################


mutable struct Chain
    body
    heads
    n
    bottleneck
    Chain(body, heads...; n=0, bottleneck=nothing) = new(body, heads..., n, bottleneck)
end

Chain(b::Body) = Chain(b, []; n=0)
(c::Chain)(h::Head...) = addHead!(c, h...)

function addHead!(c::Chain, h...)
    c.heads = vcat(c.heads..., h...)
    c.n = length(c.heads)
end

##########################################


function meanLoss(c::Chain, d::Data)
    J = [Float32(0) for n in 1:c.n]
    for (j, (x, y)) in enumerate(d)
        for (hIdx, (h, yi)) in enumerate(zip(c.heads, y))
            J[hIdx] += h.lossF(h(c.body(x)), yi)
        end
    end
    J ./ length(d)
end


every(n,itr) = (x for (i,x) in enumerate(itr) if i%n == 0)


every (generic function with 1 method)

In [ ]:

function train!(c::Chain, dTrn::Data, dTst::Data; iters=500, period=10)
    results = []
    for i=1:period:iters
        push!(
            results, (
                c(dTrn),
                c(dTst),
                1-accuracy(c; data=dTrn),
                1-accuracy(c; data=dTst)
            )
        )
        for (x, y) in every(period, dTrn)
            J = @diff c(x, y)
            for p in params(J)
                ∇p = grad(J, p)
                update!(p, ∇p)
            end
        end
    end
    push!(
        results, (
            c(dTrn),
            c(dTst),
            1-accuracy(c; data=dTrn),
            1-accuracy(c; data=dTst)
        )
    )
    results = reshape(collect(Float32,flatten(results)),(4,:))
    return 0:period:iters, results
end


train! (generic function with 1 method)

In [7]:

function meanLoss(c::Chain, d::Data)
    J = [Float32(0) for n in 1:c.n]
    for (j, (x, y)) in enumerate(d)
        for (hIdx, (h, yi)) in enumerate(zip(c.heads, y))
            J[hIdx] += h.lossF(h(c.body(x)), yi)
        end
    end
    J ./ length(d)
end


function meanAccuracy(c::Chain, d::Data)
    A = [Float32(0) for n in 1:c.n]
    for (j, (x, y)) in enumerate(d)
        for (hIdx, (h, yi)) in enumerate(zip(c.heads, y))
            A[hIdx] += sum(h(c.body(x)) .== yi)
        end
    end
    A ./ length(d)
end


meanAccuracy (generic function with 1 method)

# Model

In [ ]:

seqLen = size(xTrn, 1)

tasks = [
  "22RV1.AR-C19", "22RV1.AR-V7", "LNCaP.dht.AR", "LNCaP.veh.AR", "22RV1.HOXB13", 
  "LN95.AR-C19", "LN95.AR-V7", "LN95.HOXB13", "malignant.1.AR", "malignant.2.AR",
  "malignant.3.AR", "malignant.4.AR", "non-malignant.1.AR", "non-malignant.2.AR"
]


bpnetBody = Body(
    Conv(25,4,1,64, padding=(12, 0)) # Int(max((cx - 1) * s1 + w1 - cx, 0)/2)
)



nLayer = 2

for i=1:nLayer
    rate = 2^i
    addLayer!(bpnetBody,
        Conv(3,1,64,64, padding=(rate, 0), dilation=(rate, 0))
    )
end


bpnet = Chain(bpnetBody)


for task in tasks
    profileHead = Head(
        DeConv(25,1,2,64, padding=(12,0));
        lossF=
            multinomial_nll
    )
    println("profileHead")
    countHead = Head(
        Pool((seqLen, 1)),
        Dense(64,2, window=(seqLen,1));
        lossF=
            mse
    )
    println("countHead")
    bpnet(profileHead, countHead)
end




profileHead
countHead
profileHead
countHead
profileHead
countHead
profileHead
countHead
profileHead
countHead
profileHead
countHead
profileHead
countHead
profileHead
countHead
profileHead
countHead
profileHead
countHead
profileHead
countHead
profileHead
countHead
profileHead
countHead
profileHead
countHead


In [ ]:
meanLoss(bpnet, dTrn)

LoadError: ignored

In [ ]:

[l.nParameters for l in collect(bpnet.body.layers)]
[l.nParameters for h in collect(bpnet.heads) for l in h.layers]


28-element Array{Int64,1}:
 3200
  130
 3200
  130
 3200
  130
 3200
  130
 3200
  130
 3200
  130
 3200
    ⋮
 3200
  130
 3200
  130
 3200
  130
 3200
  130
 3200
  130
 3200
  130

In [ ]:
training = train!(bpnet, dTrn, dTst)
